# Convert community area to side area JSON object

## Load community areas

In [11]:
# load community area file
import pandas as pd

# read in the community area file
df = pd.read_csv('comarea.csv')
df.head()

,AREA_NUMBER,COMMUNITY,SIDE,GEOM
0,1,ROGERS PARK,Far North Side,"POLYGON((-87.6545639965891 41.998181358122, -8..."
1,2,WEST RIDGE,Far North Side,"POLYGON((-87.6848528132288 42.0194867077215, -..."
2,3,UPTOWN,Far North Side,"POLYGON((-87.6410325151573 41.9548318492655, -..."
3,4,LINCOLN SQUARE,Far North Side,"POLYGON((-87.6744115943986 41.9762015856515, -..."
4,5,NORTH CENTER,North Side,"POLYGON((-87.6733822119817 41.9333950059926, -..."


## Merge the community areas GEOM by SIDE

In [12]:
# import Polygon
from shapely.geometry import Polygon
from shapely.wkt import loads

# define function to merge polygons
def merge_geoms(geoms):
    merged = Polygon()
    for geom in geoms:
        merged = merged.union(loads(geom))
    return merged

# group by SIDE column and apply merge_geoms function to GEOM column
merged_df = df.groupby('SIDE')['GEOM'].apply(merge_geoms).reset_index()
print(merged_df.head(10))

                 SIDE                                               GEOM
0             Central  POLYGON ((-87.6385465883832 41.8868573755107, ...
1      Far North Side  MULTIPOLYGON (((-87.682895212355 42.0194673924...
2  Far Southeast Side  POLYGON ((-87.6340574930654 41.7415670050351, ...
3  Far Southwest Side  POLYGON ((-87.6829090011201 41.7355222386606, ...
4          North Side  POLYGON ((-87.7317475439354 41.9322886468486, ...
5      Northwest Side  POLYGON ((-87.7925823032591 41.9185174817885, ...
6          South Side  POLYGON ((-87.6433169007906 41.8496926612565, ...
7      Southwest Side  POLYGON ((-87.8012581874017 41.785667463855, -...
8           West Side  POLYGON ((-87.7399820065206 41.8659591989946, ...


In [10]:
# export to CSV
merged_df.to_csv('side_area.csv', index=False)

In [15]:
import json

# convert to a GeoJSON object
features = []
for i, (index, geom) in enumerate(merged_df.items()):
    feature = {
        "type": "Feature",
        "geometry": geom.__geo_interface__,
        "properties": {
            "name": index
        }
    }
    features.append(feature)

geojson = {
    "type": "FeatureCollection",
    "features": features
}

with open('side_area.json', 'w') as outfile:
    json.dump(geojson, outfile)

AttributeError: 'Series' object has no attribute '__geo_interface__'